In [5]:
from datetime import datetime, timedelta
print(datetime.now())

begin_date = (datetime.now() - timedelta(90)).strftime('%Y-%m-%d')
end_date = datetime.now().strftime('%Y-%m-%d')
days_per_batch = 15

2023-01-19 09:24:13.040596


In [6]:
import sys
sys.path.append('/home/maximilian.hofmann/ff_utils/src')

from importlib import reload

In [7]:
from importlib import reload
from itertools import repeat
from date_utils import partition_date_range
import pandas as pd
from multiprocessing.dummy import Pool
from multiprocessing import cpu_count

In [8]:
import functions as fu
reload(fu)

/app/infra/data/jupyterhub/images/singleuser_spark/main.binary.runfiles/vendor_python_geopandas/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


<module 'functions' from '/home/maximilian.hofmann/tools/pricing-tool-v2/data/functions.py'>

In [9]:
from ds.utilities.io import ds_trino

In [18]:
# sql = '''
# SELECT DISTINCT 
#     CAST(DATE_TRUNC('MONTH', last_seen) AS DATE) AS month,
#     super_region,
#     country_code,
#     service_slug,
#     COUNT(*)
# FROM iceberg.marketintel.storefront_latest_view
# GROUP BY 1,2,3,4
# ORDER BY 2,1,3,4
# '''

In [19]:
# data = ds_trino.fetch_data(sql)

In [20]:
# data['super_region'].unique()

In [21]:
# data[data['super_region']=='EMEA']

In [11]:
begin_date = pd.to_datetime('2022-12-01')
end_date = datetime.now().strftime('%Y-%m-%d')
days_per_batch = 5

In [7]:
super_region = 'EMEA'
# country_code = 'GBR'
service_slug = 'ubereats-marketintel'

In [2]:
with open('/home/maximilian.hofmann/tools/pricing-tool-v2/data/queries/sql_ofo_scrape_data_iceberg.sql', 'r') as f:
    sql_template = f.read()

In [3]:
project_name = 'css-operations'
stag_dataset_name = 'brand_science_stag'
stag_table_name = 'pricing_ofo_scrape_data_stag'
full_stag_table_name = '.'.join([project_name, stag_dataset_name, stag_table_name])

In [12]:
date_list = partition_date_range(pd.to_datetime(begin_date), pd.to_datetime(end_date), days_per_batch, week_trunc=False)
date_list

[['2022-12-01', '2022-12-05'],
 ['2022-12-06', '2022-12-10'],
 ['2022-12-11', '2022-12-15'],
 ['2022-12-16', '2022-12-20'],
 ['2022-12-21', '2022-12-25'],
 ['2022-12-26', '2022-12-30'],
 ['2022-12-31', '2023-01-04'],
 ['2023-01-05', '2023-01-09'],
 ['2023-01-10', '2023-01-14'],
 ['2023-01-15', '2023-01-19']]

In [13]:
clustering_fields=['country_code','month']

In [ ]:
res = []
with Pool(min(cpu_count(), len(date_list))) as p:
    r = p.starmap(fu.process_ofo_data_date_range_iceberg, zip(repeat(sql_template), date_list, repeat(super_region), repeat(service_slug), repeat(full_stag_table_name), repeat(clustering_fields)))
    res.append(r)

Start data cleaning...
Start data cleaning...
Start data cleaning...
Start data cleaning...
Start data cleaning...
Start data cleaning...


In [3]:
prod_dataset_name = 'brand_science'
prod_table_name = 'pricing_ofo_scrape_data'
full_prod_table_name = '.'.join([project_name, prod_dataset_name, prod_table_name])

In [ ]:
import time
time.sleep(60)

In [10]:
create_tbl = fu.create_tbl(full_stag_table_name, full_prod_table_name)

In [11]:
upsert_tbl = fu.upsert_tbl_ofo_scrape_data(full_stag_table_name, full_prod_table_name)